In [2]:
import backtrader as bt
import pandas as pd
import numpy as np
import datetime

# for loading algoseek data
import algoseek_connector
import algoseek_connector.functions as fn

# for algoseek user credentials
from dotenv import load_dotenv
import os

# to manage paths
import os.path
import sys

### Load Algoseek Data

#### Data Fetch

In [2]:
# load_dotenv() # load environment variables

# jhost = os.getenv('host_julian')
# juser = os.getenv('user_julian')
# jpassword = os.getenv('password_julian')

# ahost = os.getenv('host_angel')
# auser = os.getenv('user_angel')
# apassword = os.getenv('password_angel')

In [16]:
# # Instantiate the session
# session = algoseek_connector.Session(ahost, auser, apassword)
# session.ping()

# resource = algoseek_connector.DataResource(session)

# taq_min = resource.datagroups.USEquityMarketData.datasets.TradeAndQuoteMinuteBar

# volatile_tech_tickers = ['AMD']
# # volatile_tech_tickers = ['AMD', 'SQ', 'TSLA']


# taq1min = taq_min.filter(
#        (taq_min.Ticker.isin(volatile_tech_tickers)) &
#        # ('2022-07-11' >= taq_min.TradeDate <= '2022-07-12')
#        ('2022-07-11' == taq_min.TradeDate)
# ).fetch()

In [5]:
# save backup file

# os.chdir('C:/Users/angel/Documents/Documents/GitHub/AlgoSeek-Notebooks/AlgoSeekNotebooks')
# taq1min.to_csv('AlgoSeekNotebooks/datasets/taq_1min_volatile_tech_backup.csv')

#### Or Fetch Locally Stored Data

In [3]:
idx = pd.IndexSlice

In [5]:
taq_aapl = pd.read_parquet('data/us_equity/taq_min/AAPL.parquet')
taq_aapl['BarDateTime'] = pd.to_datetime(taq_aapl['BarDateTime'], format='%Y-%m-%d %H::%M:%S')

In [6]:
taq_aapl.set_index(['BarDateTime'], inplace=True)

In [50]:
# taq_aapl2 = taq_aapl['2022-01-01 4:00:00-05:00': '2022-01-14 19:59:00-05:00']

In [52]:
# taq_aapl3 = taq_aapl['2022-01-01 4:00:00-05:00': '2022-01-14 19:59:00-05:00']

In [55]:
# taq_aapl4 = taq_aapl[taq_aapl['TradeDate'][:'2022-01-01']]

In [7]:
taq_aapl = taq_aapl.iloc[0:1500]

In [8]:
taq_amzn = pd.read_parquet('data/us_equity/taq_min/AMZN.parquet')
taq_amzn['BarDateTime'] = pd.to_datetime(taq_amzn['BarDateTime'], format='%Y-%m-%d %H::%M:%S')

In [9]:
taq_amzn.set_index(['BarDateTime'], inplace=True)

In [10]:
taq_amzn = taq_amzn.iloc[0:1500]

In [9]:
taq_aapl.to_csv('AAPL_sample.csv')
taq_amzn.to_csv('AMZN_sample.csv')

#### Data Preprocessing

In [6]:
# # create backup variable for data prep
# taq1min = taq

# taq1min.TradeDate = pd.to_datetime(taq.TradeDate, format="%Y-%m-%d")
# taq1min.set_index(['BarDateTime', 'Ticker'], inplace=True)

# taq1min.sort_index().groupby('BarDateTime', group_keys=False)

# taq1min['date'] = pd.to_datetime(taq1min.index.get_level_values('BarDateTime').date)

In [ ]:
# taq.to_csv('AlgoSeekNotebooks/datasets/taq_1min_volatile_tech.csv')

### Load Data Into Backtrader

In [ ]:
class PandasData(bt.feed.DataBase):
    linesoverride = True  # discard usual OHLC structure
    # datetime must be present and last
    lines = ('Signal',)
    params = (
        ('dtformat', '%Y-%m-%d'),
        ('Date', 0),
        ('time', None),
        ('open', 1),
        ('high', 2),
        ('low', 3),
        ('close', 4),
        ('volume', 5),
        ('Signal', 6),
        ('timeframe', bt.TimeFrame.Days),
        ('compression', 1),
        ('openinterest', None)
    )

In [ ]:
from backtrader.feed import DataBase

class MyFeed(DataBase):
    def __init__(self):
        super(MyFeed, self).__init__()
        self.list = df # dataframe with date-time OHLC structure
        self.n = 0

        self.fromdate = self.list['datetime'][0] # 1st available date within data set
        self.todate = self.list['datetime'][len(self.list) - 1] # last available date within data set
        self.timeframe = bt.TimeFrame.Minutes
        print("from=%s,to=%s" % (self.fromdate, self.todate))

        self.m = {}
        # print(self.list)

    def start(self):
        # Nothing to do for this data feed type
        pass

    def stop(self):
        # Nothing to do for this data feed type
        pass

    def _load(self):
        if self.n >= len(self.list):
            return False

        r = self.list.iloc[self.n]
        self.lines.datetime[0] = date2num(r['datetime'])

        self.lines.open[0] = r['open']
        self.lines.high[0] = r['high']
        self.lines.low[0] = r['low']
        self.lines.close[0] = r['close']
        self.lines.volume[0] = r['volume']
        self.m[r['datetime']] = r

        self.n = self.n + 1
        return True

In [16]:
from backtrader.feed import CSVDataBase

class GenericCSVData(CSVDataBase):
    params = (
        ('nullvalue', float('NaN')),
        ('dtformat', '%Y-%m-%d %H::%M:%S'), # %f%z - may need to add for UTC
        # ('tmformat', '%H:%M:%S'),

        ('datetime', None), # index contains datetime
        # ('time', -1), # ???
        ('open', 9),
        ('high', 18),
        ('low', 27),
        ('close', 35),
        ('volume', 48)#,
        # ('openinterest', 6)
    )

### Strategy Logic

#### Bollinger Band Mean Reversion

- Buy when standard deviation <= -2
- Sell when standard deviation >= 2
- Liquidate when standard deviation <= 0.5 or 0

In [ ]:
class MyStrategy(bt.Strategy):
    lines = ('signal',)
    params = (('p1', 5), ('p2', 30),)

    def __init__(self):
        sma1 = bt.indicators.SMA(period=self.p.p1)
        sma2 = bt.indicators.SMA(period=self.p.p2)
        self.lines.signal = sma1 - sma2

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])


'''
def next(self):
    for i, d in enumerate(self.datas):
        if not self.getposition(d).size:
            if self.crossovers[i] > 0: 
                self.buy(data = d)
        elif self.crossovers[i] < 0: 
            self.close(data = d)
'''

In [ ]:
'''

Strategy Pseudocode



'''

### Run Cerebro Engine

In [1]:
if __name__ == '__main__':

    cerebro = bt.Cerebro()

    # strats = cerebro.optstrategy(MeanReversion)

    # data = bt.feeds.YahooFinanceCSVData(
    #     dataname=datapath,
    #     # Do not pass values before this date
    #     fromdate=datetime.datetime(2000, 1, 1),
    #     # Do not pass values before this date
    #     todate=datetime.datetime(2000, 12, 31),
    #     # Do not pass values after this date
    #     reverse=False)

    datalist = [
        ('FAANG_data/us_equity/taq_min/AAPL_sample.csv', 'AAPL'),
        ('FAANG_data/us_equity/taq_min/AMZN_sample.csv', 'AMZN')
    ]

    for i in range(len(datalist)):
        data = GenericCSVData(dataname=datalist[i][0])
        cerebro.adddata(data, name=datalist[i][1])

    # modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    # datapath = os.path.join(modpath, 'AlgoSeekNotebooks/datasets/taq_1min_volatile_tech.csv')

    # # Create a Data Feed
    # data = bt.feeds.CSVDataBase(
    #     dataname=datapath,
    #     reverse=False)

    # Add the Data Feed to Cerebro
    # cerebro.adddata(data)

    # Set our desired cash start
    startcash = 100000.0
    cerebro.broker.setcash(startcash)

    # Add a FixedSize sizer according to the stake
    # cerebro.addsizer(bt.sizers.FixedSize, stake=10)

    # Set the commission
    cerebro.broker.setcommission(commission=0.0)

    # Run strategy
    # cerebro.run(maxcpus=1)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    cerebro.run()

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

    portvalue = cerebro.broker.getvalue()
    pnl = portvalue - startcash

    #Print out the final result
    print('Final Portfolio Value: ${}'.format(portvalue))
    print('P/L: ${}'.format(pnl))

    #Finally plot the end results
    cerebro.plot(style='candlestick')

NameError: name 'bt' is not defined

### Benchmark Strategy

- Buy and hold each company with equal weighted portfolio allocation